# Welcome to dfsql!

[dfsql](https://github.com/mindsdb/dfsql) allows you to query Pandas and Modin DataFrames using SQL queries.

This notebook will guide you through the main features.

In [ ]:
import os
import pandas
import pandas as pd

In [ ]:
import warnings
# Filter Modin userwarnings about defaulting to pandas implementations
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Comment out if you don't have modin installed
%env USE_MODIN=True
import modin.pandas as pd
from dfsql.config import Configuration
Configuration.as_dict()

In [ ]:
from dfsql import DataSource

In [ ]:
%load_ext autoreload
%autoreload 2

# Loading data

In [ ]:
os.makedirs('testdrive_csvs', exist_ok=True)

In [ ]:
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore.csv -o testdrive_csvs/googleplaystore.csv
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore_user_reviews.csv -o testdrive_csvs/googleplaystore_user_reviews.csv

## The dataset contains google play apps and user reviews

In [ ]:
googleplaystore = pd.read_csv('testdrive_csvs/googleplaystore.csv')
googleplaystore.head()

In [ ]:
googleplaystore_user_reviews = pd.read_csv('testdrive_csvs/googleplaystore_user_reviews.csv')
googleplaystore_user_reviews.head()

In [ ]:
googleplaystore.Price

# Quick interfaces

## sql_query

`sql_query` takes as input the sql string and Pandas or Modin dataframes as keywords.

In [ ]:
from dfsql import sql_query

In [ ]:
result_df = sql_query("SELECT App, Category, Rating, Price FROM table1 WHERE Price = \"0\" LIMIT 5", 
                      table1=googleplaystore)
result_df

Multiple tables can be passed to perform joins and complex queries.

In [ ]:
sql = """
SELECT App, avg(Sentiment_Polarity) as avg_sentiment_polarity
FROM (
    SELECT App, Sentiment, CAST(Sentiment_Polarity as float) as Sentiment_Polarity
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews 
    ON googleplaystore.App = googleplaystore_user_reviews.App 
) AS sub
GROUP BY App
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
result_df = sql_query(sql, googleplaystore=googleplaystore, googleplaystore_user_reviews=googleplaystore_user_reviews)
result_df

## Pandas/Modin extensions


`dataskillet.extensions` adds the `.sql` method to Pandas and Modin dataframes when imported. 

This method takes only an sql query. It's useful when only one table is used in the query.

In [ ]:
# Make sure to import it **AFTER** Pandas or Modin
import dfsql.extensions

### Example with a Modin DataFrame

In [ ]:
print(type(googleplaystore))
googleplaystore.sql("SELECT App, Category") # Alternatively: "SELECT App, Category FROM temp"

### Example with a Pandas DataFrame

The operations are still executed in Modin if it's enabled.

In [ ]:
googleplaystore_pandas = pandas.read_csv('testdrive_csvs/googleplaystore.csv')

print(type(googleplaystore_pandas))
googleplaystore_pandas.sql("SELECT App, Category")

# In-depth usage

Under the hood `sql_query` creates a DataSource, adds tables to it, executes the query, and then tears everything down. This is not very efficient if many queries have to be executed against the same tables. In that case it's better to use DataSource directly.

DataSource is the analogus of database. It stores information about tables, loads tables from disk when they are needed, handles caching. 
It accepts queries like a database would.

## Creating the DataSource
On creation, we need to specify a directory to store table metadata for the DataSource.

In [ ]:
metadata_dir = 'testdrive_metadata'
os.mkdir(metadata_dir)

In [ ]:
ds = DataSource(metadata_dir=metadata_dir)

The metadata dir now stores information about tables. However we currently have no tables. Lets create some

In [ ]:
ds.query('SHOW TABLES')

## Creating tables

A `Table` is an abstraction over a dataframe. It loads the dataframe on-demand. On creation it applies simple preprocessing by converting dtypes.

In [ ]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore.csv")')

In [ ]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore_user_reviews.csv")')

In [ ]:
ds.query('SHOW TABLES')

Now that we have some tables, the information about them is stored in metadata. 

If we recreate the datasource using the same `metadata_dir`, **we don't need to add the tables again**.

In [ ]:
ds = DataSource(metadata_dir=metadata_dir)

In [ ]:
ds.query('SHOW TABLES')

If we need it, we can clear the metadata and recreate the DataSource using `DataSource.create_new(metadata_dir)`

## Querying

In [ ]:
ds.query('SELECT * FROM googleplaystore LIMIT 5')

In [ ]:
ds.query("SELECT App FROM googleplaystore LIMIT 5")

In [ ]:
ds.query("SELECT CAST(Price as int) as price_int FROM googleplaystore WHERE Price = \"0\" LIMIT 5")

In [ ]:
ds.query("SELECT App, Category, `Content Rating` FROM googleplaystore WHERE Price = \"0\"")

In [ ]:
ds.query("SELECT App, Category, Reviews FROM googleplaystore WHERE Category = \"FAMILY\" AND Price = \"0\" ORDER BY App ASC LIMIT 100")

In [ ]:
ds.query("SELECT App, Sentiment, Sentiment_Polarity FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.App = googleplaystore_user_reviews.App LIMIT 10")

In [ ]:
sql = """
SELECT App, avg(Sentiment_Polarity) as avg_sentiment_polarity
FROM (
    SELECT App, Sentiment, CAST(Sentiment_Polarity as float) as Sentiment_Polarity
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.App = googleplaystore_user_reviews.App 
) AS sub
GROUP BY App
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
ds.query(sql)

In [ ]:
sql = """
SELECT count(App) as count_apps
FROM (
    SELECT App
    FROM googleplaystore 
) AS sub
"""
ds.query(sql)

In [ ]:
sql = """
SELECT Category, count(App) as count_apps
FROM (
    SELECT Category, App FROM googleplaystore 
) AS sub
GROUP BY Category
ORDER BY count_apps DESC
LIMIT 10
"""
ds.query(sql)

In [ ]:
sql = """
SELECT count(Category) as uniq_categories
FROM (
    SELECT DISTINCT Category FROM googleplaystore 
) AS sub
"""
ds.query(sql)

In [ ]:
sql = """
SELECT count(distinct Category) as uniq_categories
FROM googleplaystore 
"""
ds.query(sql)

In [ ]:
tdf = ds.query("SELECT App || '_app_name'   FROM googleplaystore LIMIT 5")
tdf[0]

In [ ]:
ds.query("SELECT upper(App), Category FROM googleplaystore WHERE App LIKE \".*best.*\" LIMIT 5")

In [ ]:
ds.query("SELECT App, Sentiment, Translated_Review FROM googleplaystore_user_reviews WHERE Translated_Review IS NULL")

In [ ]:
ds.query("SELECT App, Sentiment FROM googleplaystore_user_reviews WHERE Sentiment IN ('Positive', 'Negative')")

## Dataframe caching

Dataskillet caches dataframes in memory to speed up queries against the same tables.

First let's clear the cache

In [ ]:
ds.cache.clear()

Querying with the cache clear

In [ ]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

Check the cache

In [ ]:
ds.cache.get.cache_info()

Querying with the cache filled

In [ ]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

Ensuring it was retrieved from cache:

In [ ]:
ds.cache.get.cache_info()

In [ ]:
%%timeit
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')